In [1]:
# Author ~ Markela Zeneli
# Goldsmiths, University of London
########################################

import pandas as pd
import os
import fnmatch
import datetime
import math

race = []
gender = []
outcome = []
age = []
crime = []

count1 = 0
count2 = 0
count3 = 0
count4 = 0

#Add CSV columns to respective lists
def all(path):
    global race
    global gender
    global outcome
    global age
    
    df = pd.read_csv(path, sep=',')

    race += df['Officer-defined ethnicity'].tolist()
    gender +=df['Gender'].tolist()
    outcome += df['Outcome'].tolist()
    age += df['Age range'].tolist()

def crimeCount(path):
    global crime

    df = pd.read_csv(path, sep=',')

    crime += df['Crime type'].tolist()

def getDates(month1, year1, month2, year2):

    if month1==month2 and year1==year2:
        if month1<10:
            #Populate the Stop and Search demographics lists for Metropolitan and City of London
            all('data/'+str(year1)+'-0'+str(month1)+'/'+str(year1)+'-0'+str(month1)+'-city-of-london-stop-and-search.csv')
            all('data/'+str(year1)+'-0'+str(month1)+'/'+str(year1)+'-0'+str(month1)+'-metropolitan-stop-and-search.csv')
            
            #Populate the Crimes Reported list for Metropolitan and City of London
            crimeCount('data/'+str(year1)+'-0'+str(month1)+'/'+str(year1)+'-0'+str(month1)+'-city-of-london-street.csv')
            crimeCount('data/'+str(year1)+'-0'+str(month1)+'/'+str(year1)+'-0'+str(month1)+'-metropolitan-street.csv')
        else:
            all('data/'+str(year1)+'-'+str(month1)+'/'+str(year1)+'-'+str(month1)+'-city-of-london-stop-and-search.csv')
            all('data/'+str(year1)+'-'+str(month1)+'/'+str(year1)+'-'+str(month1)+'-metropolitan-stop-and-search.csv')
            crimeCount('data/'+str(year1)+'-'+str(month1)+'/'+str(year1)+'-'+str(month1)+'-city-of-london-street.csv')
            crimeCount('data/'+str(year1)+'-'+str(month1)+'/'+str(year1)+'-'+str(month1)+'-metropolitan-street.csv')

    #Start date
    d = datetime.date(year1,month1,15)

    #Size of each step
    month_delta = datetime.timedelta(days=30)

    #End date
    end_date = datetime.date(year2,month2,15)
    #print(end_date)

    #Calculates difference in dates (by metric of .days), and turns into months by dividing by 30 and rounding up
    for i in range(math.ceil(((end_date - d).days)/30)):
        #File paths for months with 1 digit (adds a 0 in front to keep in line with format of file path)
        if (d + i*month_delta).month < 10:
            #Populate the Stop and Search demographics lists for Metropolitan and City of London (adds a zero in front if month is a single digit)
            all('data/'+str((d + i*month_delta).year)+'-0'+str((d + i*month_delta).month)+'/'+str((d + i*month_delta).year)+'-0'+str((d + i*month_delta).month)+'-city-of-london-stop-and-search.csv')
            all('data/'+str((d + i*month_delta).year)+'-0'+str((d + i*month_delta).month)+'/'+str((d + i*month_delta).year)+'-0'+str((d + i*month_delta).month)+'-metropolitan-stop-and-search.csv')
            #Populate the Crimes Reported list for Metropolitan and City of London (adds a zero in front if month is a single digit)
            crimeCount('data/'+str((d + i*month_delta).year)+'-0'+str((d + i*month_delta).month)+'/'+str((d + i*month_delta).year)+'-0'+str((d + i*month_delta).month)+'-city-of-london-street.csv')
            crimeCount('data/'+str((d + i*month_delta).year)+'-0'+str((d + i*month_delta).month)+'/'+str((d + i*month_delta).year)+'-0'+str((d + i*month_delta).month)+'-metropolitan-street.csv')
        else:
            #Populate lists if months >= 10
            all('data/'+str((d + i*month_delta).year)+'-'+str((d + i*month_delta).month)+'/'+str((d + i*month_delta).year)+'-'+str((d + i*month_delta).month)+'-city-of-london-stop-and-search.csv')
            all('data/'+str((d + i*month_delta).year)+'-'+str((d + i*month_delta).month)+'/'+str((d + i*month_delta).year)+'-'+str((d + i*month_delta).month)+'-metropolitan-stop-and-search.csv')
            crimeCount('data/'+str((d + i*month_delta).year)+'-'+str((d + i*month_delta).month)+'/'+str((d + i*month_delta).year)+'-'+str((d + i*month_delta).month)+'-city-of-london-street.csv')
            crimeCount('data/'+str((d + i*month_delta).year)+'-'+str((d + i*month_delta).month)+'/'+str((d + i*month_delta).year)+'-'+str((d + i*month_delta).month)+'-metropolitan-street.csv')

In [2]:
#Time range of CSVs
month1=12
year1=2017
month2=11
year2=2020
#Increment placeholder
month3=0
year3=year1

#This function iterates through the timescale of the dataset at varying increments. It will call the "getDates" function at each increment, process the statistics in those lists using for loops, and print the statistics before moving onto the next increment.

def dateIterator(increment):
    global month1, year1, month2, year2, month3, year3, race, age, gender, outcome, crime, count1, count2, count3, count4

    #While the start of each increment is befor the end date
    while ((month1 <= month2 and year1 <= year2) or (month2 < month1 and year1 < year2)):

        #Add increment to a temp month, "month3". This is the temporary end date.
        month3=month1+increment-1

        #If temp month exceeds 12 (i.e goes into the next year), update temp date accordingly
        if month3 > 12:
            year3 = year1 + 1
            month3 = month3 % 12

        #If temporary end date date exceeds final end date, run "getDates" with final end date instead, then quit the script
        if ((month2 < month3 and year2 <= year3) or (month3 < month2 and year2 < year3)):
            getDates(month1, year1, month2, year2)
            quit()
            
        #Call "getDates" with start date and temporary end date
        getDates(month1, year1, month3, year3)

        #Stop and search demographics stat
        for i in range(len(race)):
            if race[i] == "Black" and gender[i] == "Male" and (age[i] == "10-17" or age[i] == "18-24"):
                count1+=1
                if outcome[i] == "A no further action disposal"  :
                    count2+=1
        #"""+geography=fixed for count2"""

        stat = count2/count1

        #Crime count stat * see KEY in markdown table below
        for i in range(len(crime)):
            if crime[i] != "Anti-social behaviour":
                count3+=1
        
        count4 = len(crime)

        stat_c = count3/count4


        #Print stats for this increment
        print("--------------------------------------")
        print("Start date: "+str(month1)+"/"+str(year1))
        print("End date: "+str(month3)+"/"+str(year3))
        print("Stop and Search")
        print(str(count1) + ',' + str(count2))
        print(stat)
        print("Crime")
        print(str(count3) + ',' + str(count4))
        print(stat_c)
        print("--------------------------------------")

        #Reset lists and variables
        race = []
        gender = []
        outcome = []
        age = []
        crime = []
        count1 = 0
        count2 = 0
        count3 = 0
        count4 = 0

        #Check if new start date (month3+1) exceeds 12 month scale, and update accordingly if so
        if month3+1 > 12:
            year3 = year1 + 1
            month3 = (month3 % 12) + 1
        
        #Update start date to new values
        month1 = month3+1
        year1 = year3

#Call dateIterator with chosen increment
dateIterator(2)

--------------------------------------
Start date: 12/2017
End date: 1/2018
Stop and Search
21689,8472
0.39061275300843745
Crime
134974,166219
0.8120250994170342
--------------------------------------
--------------------------------------
Start date: 2/2018
End date: 3/2018
Stop and Search
10851,7555
0.6962491936227075
Crime
62536,76616
0.8162263756917615
--------------------------------------
--------------------------------------
Start date: 4/2018
End date: 5/2018
Stop and Search
12970,9407
0.7252891287586739
Crime
67036,86981
0.7706970487807682
--------------------------------------
--------------------------------------
Start date: 6/2018
End date: 7/2018
Stop and Search
10673,7690
0.7205096973671883
Crime
70829,90229
0.7849915215728868
--------------------------------------
--------------------------------------
Start date: 8/2018
End date: 9/2018
Stop and Search
27046,19729
0.7294609184352584
Crime
138729,176570
0.7856883955371807
--------------------------------------
--------

## Table of stats

KEY: 

(*) - This probability is the number of street crimes reported which are not related to "Anti-social behaviour". This constraint exists because "Anti-social behaviour" can be subjective to the person reporting the crime, and does not always indicate a threat to society. At least, the type of threat which supposedly justifies the volume of Stop and Searches that do occur. This constraint can also be seen as an acknowledgement of the fact that Stop and Searches (and section 60 in particular) are seen to have a deterrence effect on crime. ["The police use these powers to help make the local community safer by preventing and detecting crime."](https://www.met.police.uk/advice/advice-and-information/st-s/stop-and-search/stop-and-search-process/). 


TBC

|start date|end date|S+S innocence|S+S volume|Crime*|Crime reported|notes|
|---|---|---|---|---|---|---| 
|12/2017|11/2020|0.76|66245|0.73|2634530|entire dataset, no constraints|
|12/2017|01/2018|0.39|21689|0.81|85004|general stats for pairs of months|
|64|64|10|1152|1.0914|1866641|10-11/2020|
|64|64|10|1152|1.0914|2316490||
|64|64|10|1152|1.0914|3060919|end pairs of months|
|64|64|10|1152|1.0914|0.7320|pairs of months, ethnicity = White, age = 10-24|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|pairs of months, ethnicity = White, age = 25+|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|pairs of months, ethnicity = Asian, age = 10-24|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|pairs of months, ethnicity = Asian, age = 25+|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|pairs of months, ethnicity = Black, age = 10-24|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|pairs of months, ethnicity = Black, age = 25+|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|general stats for every 3 months|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every 3 months, ethnicity = White, age = 10-24|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every 3 months, ethnicity = White, age = 25+|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every 3 months, ethnicity = Asian, age = 10-24|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every 3 months, ethnicity = Asian, age = 25+|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every 3 months, ethnicity = Black, age = 10-24|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every 3 months, ethnicity = Black, age = 25+|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|general stats for every 6 months|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every 6 months, ethnicity = White, age = 10-24|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every 6 months, ethnicity = White, age = 25+|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every 6 months, ethnicity = Asian, age = 10-24|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every 6 months, ethnicity = Asian, age = 25+|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every 6 months, ethnicity = Black, age = 10-24|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every 6 months, ethnicity = Black, age = 25+|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|general stats for every year|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every year, ethnicity = White, age = 10-24|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every year, ethnicity = White, age = 25+|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every year, ethnicity = Asian, age = 10-24|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every year, ethnicity = Asian, age = 25+|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every year, ethnicity = Black, age = 10-24|
|64|64|10|1152|1.0914|0.7320||
|64|64|10|1152|1.0914|0.7320|every year, ethnicity = Black, age = 25+|
|64|64|10|1152|1.0914|0.7320||
